In [1]:
import pandas as pd
import numpy as np

In [2]:
RegSD = pd.read_csv('RegularSeasonDetailedResults.csv')
RegSD.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set_style('darkgrid')
matplotlib.rc('font', size=10)
matplotlib.rc('axes', titlesize=10)
matplotlib.rc('axes', labelsize=10)
matplotlib.rc('xtick', labelsize=10)
matplotlib.rc('ytick', labelsize=10)
matplotlib.rc('legend', fontsize=10)
matplotlib.rc('figure', titlesize=10)
%matplotlib inline

In [7]:
df1 = pd.DataFrame()
df1[["team1","team2"]] = RegSD[["Wteam","Lteam"]].copy()
df1['pred'] = 1

df2 = pd.DataFrame()
df2[["team1","team2"]] = RegSD[["Lteam","Wteam"]].copy()
df2['pred'] = 0

final1 = pd.concat((df1,df2), axis=0)
n = final1["team1"].nunique()
n

355

In [8]:
unique_team = {t:i for i, t in enumerate(final1.team1.unique())}
final1["team1"] = final1["team1"].apply(lambda x: unique_team[x])
final1["team2"] = final1["team2"].apply(lambda x: unique_team[x])
final1.head()

,team1,team2,pred
0,0,67,1
1,1,164,1
2,2,93,1
3,3,221,1
4,4,110,1
